In [ ]:
from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
import os
import pandas as pd
import numpy as np
from snowflake import connector
from dotenv import load_dotenv
from pprint import pprint
import sys
from snowflake.connector.pandas_tools import write_pandas
from snowflake.snowpark import functions as F

In [ ]:
load_dotenv()

In [ ]:
account_name=os.environ.get('account_name')
user_name=os.environ.get('user_name')
password=os.environ.get('password')
database=os.environ.get('database')
schema=os.environ.get('schema')

In [ ]:
os.environ['SNOWFLAKE_ACCOUNT'] = account_name
os.environ['SNOWFLAKE_USER'] = user_name
os.environ['SNOWFLAKE_PASSWORD'] = password
os.environ['SNOWFLAKE_DATABASE'] = database
os.environ['SNOWFLAKE_SCHEMA'] = schema

In [ ]:
conn=connector.connect(
        account=account_name,
        user=user_name,
        password=password,
        database=database,
        schema=schema
    )

In [ ]:
query="""
SELECT 
    dname,
    SUM(sal),
FROM 
    test.public.emp
        JOIN
    dept
ON
     emp.deptno=dept.deptno
WHERE
    dname!='Finance'
GROUP BY
    dname
ORDER BY 
    dname;
"""

In [ ]:
df=pd.read_sql(sql=query,con=conn)

In [ ]:
df

In [ ]:
session=Session.builder.configs(SnowflakeLoginOptions("test_conn")).create()

Using Pandas Dataframe 

In [ ]:
emps=session.table("emp").to_pandas()[['DEPTNO','SAL']]
depts=session.table("dept").to_pandas()[['DEPTNO','DNAME']]

In [ ]:
df=pd.merge(left=emps,right=depts,on='DEPTNO',how='inner')[['DNAME','SAL']]

In [ ]:
df

In [ ]:
df=df.loc[~(df['DNAME']=='Finance'),:]

In [ ]:
df

In [ ]:
df=df.groupby(['DNAME']).sum()
df=df.sort_values("DNAME")
print(df)

Using Pyspark Dataframe

In [ ]:
emps=session.table("EMP").select("DEPTNO","SAL")
deps=session.table("DEPT").select("DEPTNO","DNAME")

In [ ]:
emps.show()

In [ ]:
deps.show()

In [ ]:
q=emps.join(deps, emps["DEPTNO"]==deps["DEPTNO"])
q=q.filter(q.DNAME!='Finance')
q=q.select("DNAME","SAL").group_by("DNAME").agg({"SAL":"sum"}).sort('DNAME')
q.show()

In [ ]:
pprint(q.queries)

Feature Engineering using Pandas Dataframe

In [ ]:
query="""
SELECT 
    *
FROM 
    test.public.housing;
"""

In [ ]:
df=pd.read_sql(sql=query,con=conn)

In [ ]:
size=np.round(sys.getsizeof(df)/(1024**2),2)
print(f"Memory: {size} MB")

In [ ]:
display(df)

In [ ]:
df.head()

In [ ]:
filter=df['OCEAN_PROXIMITY'].isin(values=['INLAND','ISLAND','NEAR BAY'])
df=df.loc[(filter),:]
display(df)

In [ ]:
df.columns

In [ ]:
df=df.groupby(['OCEAN_PROXIMITY'])['HOUSEHOLDS'].mean()



In [ ]:
df=df.to_frame()
df

In [ ]:
df.rename(columns={'HOUSEHOLDS':'AVG_HOUSEHOLD'},inplace=True)
df

In [ ]:
write_pandas(conn,
             df,'test.public.housing_pandas',
             auto_create_table=True,
             overwrite=True)

Feature Engineering using Snowpark Dataframe

In [ ]:
query="""
SELECT 
    CURRENT_WAREHOUSE(),
    CURRENT_DATABASE(),
    CURRENT_SCHEMA();
"""
print(session.sql(query).collect())

In [ ]:
df=session.table('HOUSING')
size=np.round(sys.getsizeof(df)/(1024.0**2),2)
print(f"Memory: {size} MB")  # Table will not be loaded in memory

In [ ]:
df.show()

In [ ]:
df.queries

In [ ]:
df.schema

In [ ]:
df=df.with_column(
        'BEDROOM_RATIO',
        F.col('TOTAL_BEDROOMS')/F.col('TOTAL_ROOMS')
)
df.show()

In [ ]:
df=df.drop('BEDROOM_RATIO')
df.show()

In [ ]:
df=df.filter(F.col('OCEAN_PROXIMITY').in_(['INLAND','ISLAND','NEAR BAY']))

In [ ]:
df.show()

In [ ]:
df=df.group_by(['OCEAN_PROXIMITY']).agg([F.avg('HOUSEHOLDS').as_(alias='AVG_HOUSEHOLDS')])

df.show()

In [ ]:
df=df.sort(F.col('AVG_HOUSEHOLDS').asc())
df.show()

In [ ]:
df.write.mode("overwrite").save_as_table("HOUSING_SNOWPARK")

In [ ]:
dfp=df.to_pandas()
dfp